In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation

# Step 1: Load and preprocess the dataset
data = pd.read_csv('optical_injection_locking.csv')
X = data['Frequency_Detuning'].values.reshape(-1, 1)  # Reshape to a 2D array
y1 = data['Temperature'].values.reshape(-1, 1)
y2 = data['Laser_Current'].values.reshape(-1, 1)
y3 = data['Bias_Vottage_of_Slave'].values.reshape(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

# Standardize the features
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
y1_train = scaler_y.fit_transform(y1_train)
y2_train = scaler_y.fit_transform(y2_train)
y3_train = scaler_y.fit_transform(y3_train)

# Step 2: Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Step 3: Train the neural network
model1 = NeuralNetwork()
model2 = NeuralNetwork()
model3 = NeuralNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
optimizer = optim.Adam(model2.parameters(), lr=0.001)
optimizer = optim.Adam(model3.parameters(), lr=0.001)

epochs1 = 1000
for epoch in range(epochs1):
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(y1_train, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model1(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs1}], Loss: {loss.item():.4f}')

epochs2 = 1000
for epoch in range(epochs2):
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(y2_train, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model2(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs2}], Loss: {loss.item():.4f}')

epochs3 = 1000
for epoch in range(epochs3):
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(y3_train, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model3(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs3}], Loss: {loss.item():.4f}')

torch.save(model1.state_dict(), "model1.pth")
torch.save(model2.state_dict(), "model2.pth")
torch.save(model3.state_dict(), "model3.pth")

# Step 4: Provide a function to get predictions based on user input for frequency detuning
def get_prediction1(freq_detuning):
    scaled_input = scaler_X.transform([[freq_detuning]])
    with torch.no_grad():
        prediction = model1(torch.tensor(scaled_input, dtype=torch.float32)).numpy()
    prediction = scaler_y.inverse_transform(prediction)
    return prediction[0][0]

def get_prediction2(freq_detuning):
    scaled_input = scaler_X.transform([[freq_detuning]])
    with torch.no_grad():
        prediction = model2(torch.tensor(scaled_input, dtype=torch.float32)).numpy()
    prediction = scaler_y.inverse_transform(prediction)
    return prediction[0][0]

def get_prediction3(freq_detuning):
    scaled_input = scaler_X.transform([[freq_detuning]])
    with torch.no_grad():
        prediction = model3(torch.tensor(scaled_input, dtype=torch.float32)).numpy()
    prediction = scaler_y.inverse_transform(prediction)
    return prediction[0][0]

# Example usage
user_freq_detuning = float(input("Enter the frequency detuning value: "))
predicted_temperature = get_prediction1(user_freq_detuning)
predicted_current = get_prediction2(user_freq_detuning)
predicted_voltage = get_prediction3(user_freq_detuning)
print(f"Predicted temperature: {predicted_temperature:.2f} °C")
print(f"Predicted current: {predicted_current:.2f}")
print(f"Predicted Bias_Vottage_of_Slave: {predicted_voltage:.2f} V")